In [ ]:
import pathlib
import sys

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
src_dir = "../../src"

In [ ]:
sys.path.append(src_dir)

In [ ]:
from package.constants import *
from package.datasets import *
from package.feature_extraction import *
from package.preprocessing import *
from package.utils import *

In [ ]:
interim = load_interim()

In [ ]:
%%time
# Extract features
create_aggregate_features(interim, level_ids[1:11], raw_numerical_features)
create_calendar_features(interim, parse_dates)
create_expanding_features(interim, level_ids[11:], raw_numerical_features)
create_pct_change_features(interim, level_ids[11], raw_numerical_features, periods)
create_scaled_features(interim, level_ids[11:], raw_numerical_features)
create_shift_features(interim, level_ids[11], [target], periods)
create_rolling_features(interim, level_ids[11:], shift_features, windows)
create_days_since_release(interim)
create_event_name(interim)
create_event_type(interim)
# See https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/144842
create_is_working_day(interim)
# See https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/150955
create_snap(interim)

In [ ]:
# Transform target
interim[transformed_target] = interim[target] * interim["sell_price"]

In [ ]:
interim.drop(
    columns=[
        "event_name_1",
        "event_name_2",
        "event_type_1",
        "event_type_2",
        "month",
        "snap_CA",
        "snap_TX",
        "snap_WI",
        "wday",
        "weekday",
        "year",
        "wm_yr_wk",
    ],
    inplace=True,
)

In [ ]:
label_encode(interim, categorical_features)

In [ ]:
reduce_memory_usage(interim, allow_float16=False)

In [ ]:
interim.info()

In [ ]:
processed_dir_path.mkdir(parents=True, exist_ok=True)
interim.to_parquet(processed_path)